In [1]:
import numpy as np
import cv2
import os
import bck_adq
from motpy import Detection, MultiObjectTracker
from csv import writer 

In [3]:
def fixColor(image):
    return(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [2]:
from deep_sort_realtime.deepsort_tracker import DeepSort

In [4]:
#path='trial.mp4'
path='pichilemu\pichilemu6224201-2.mp4'
video_stream= cv2.VideoCapture(path)
total_frames=video_stream.get(cv2.CAP_PROP_FRAME_COUNT)

In [22]:
from deep_sort_realtime.deepsort_tracker import DeepSort
tracker = DeepSort(max_age=5)
bbs = object_tracking.detect(frame) 
tracks = tracker.update_tracks(bbs, frame=frame) # bbs expected to be a list of detections, each in tuples of ( [left,top,w,h], confidence, detection_class )
for track in tracks:
    if not track.is_confirmed():
        continue
    track_id = track.track_id
    ltrb = track.to_ltrb()

AttributeError: 'function' object has no attribute 'detect'

In [26]:
def fixColor(image):
    return(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [27]:
#path='trial.mp4'
path='pichilemu\pichilemu6224201-2.mp4'
video_stream= cv2.VideoCapture(path)
total_frames=video_stream.get(cv2.CAP_PROP_FRAME_COUNT)

In [28]:
total_frames

8689.0

In [6]:
#Create folder for the result
folderpath=path.split('.')[0]+' Dataset_v2'
if not os.path.isdir(folderpath):
   os.makedirs(folderpath)

In [7]:
grayMedianFrame= bck_adq.grayFrame(path)

In [ ]:
from __future__ import print_function
import sys
import cv2
from random import randint
 
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
 
def createTrackerByName(trackerType):
  # Create a tracker based on tracker name
  if trackerType == trackerTypes[0]:
    tracker = cv2.TrackerBoosting_create()
  elif trackerType == trackerTypes[1]:
    tracker = cv2.TrackerMIL_create()
  elif trackerType == trackerTypes[2]:
    tracker = cv2.TrackerKCF_create()
  elif trackerType == trackerTypes[3]:
    tracker = cv2.TrackerTLD_create()
  elif trackerType == trackerTypes[4]:
    tracker = cv2.TrackerMedianFlow_create()
  elif trackerType == trackerTypes[5]:
    tracker = cv2.TrackerGOTURN_create()
  elif trackerType == trackerTypes[6]:
    tracker = cv2.TrackerMOSSE_create()
  elif trackerType == trackerTypes[7]:
    tracker = cv2.TrackerCSRT_create()
  else:
    tracker = None
    print('Incorrect tracker name')
    print('Available trackers are:')
    for t in trackerTypes:
      print(t)
 
  return tracker

In [46]:
tracker = DeepSort(max_age=5)
bbs = object_tracking.detect(total_frames) 
tracks = tracker.update_tracks(bbs, frame=total_frames) # bbs expected to be a list of detections, each in tuples of ( [left,top,w,h], confidence, detection_class )
for track in tracks:
    if not track.is_confirmed():
        continue
    track_id = track.track_id

NameError: name 'Object_tracking' is not defined

In [8]:
model_spec = {
        'order_pos': 1, 'dim_pos': 2, # position is a center in 2D space; under constant velocity model
        'order_size': 0, 'dim_size': 2, # bounding box is 2 dimensional; under constant velocity model
        'q_var_pos': 1000., # process noise
        'r_var_pos': 0.1 # measurement noise
    }
tracker = DeepSort(max_age=5)

In [13]:
frameCnt=0
data=[]
try:
    while(frameCnt < total_frames-1):
        ret, frame = video_stream.read()
        #% of conversion
        frameCnt+=1
        print('Process: '+str(frameCnt/total_frames*100)+'% ',end="\r")

        # Convert current frame to grayscale
        gframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate absolute difference of current frame and the median frame
        dframe = cv2.absdiff(gframe, grayMedianFrame)

        # Gaussian
        blurred = cv2.GaussianBlur(dframe, (11, 11), 0)

        #Thresholding to binarise
        ret, tframe= cv2.threshold(blurred,0,255,
                                   cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        #Identifying contours from the threshold
        (cnts, _) = cv2.findContours(tframe.copy(), 
                                     cv2.RETR_EXTERNAL, cv2 .CHAIN_APPROX_SIMPLE)


        if len(cnts) < 100:
            #For each contour draw the bounding bos
            for cnt in cnts:
                x,y,w,h = cv2.boundingRect(cnt)
                tracker.step(detections=[Detection(box=np.array([int(x-w),int(y-h),int(x+2*w),int(y+2*h)]))])
                tracks = tracker.active_tracks()
                #cv2.rectangle(frame,(int(x-w),int(y-h)),(int(x+2*w),int(y+2*h)),(0,255,0),2)
                #cv2.circle(frame, (int((x+x+w)/2),int((y+y+h)/2)),5,(0,0,255),5)
                if len(tracks)>0:
                    #cv2.putText(frame, tracks[0][0], (int(x-w),int(y-h-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                    
                    #['Frame','id','xmin','xmax','ymin','ymax']
                    data.append([frameCnt,tracks[0][0],int(x-w), int(x+2*w),int(y-h),int(y+2*h)])
            #Convert frames into png files
            cv2.imwrite(os.path.join(folderpath,str(frameCnt)+'.png'),frame)


        #Create video 
        #writer.write(cv2.resize(frame, (640,480)))

    #Release video object
    #video_stream.release()
    #writer.release()
except:
    pass

In [ ]:
pd.read_csv('KITAI_Dataset_2/test/_annotations.csv')

In [ ]:
file= open(os.path.join(folderpath,'Data.csv'), 'w') 
writer = writer(file)
writer.writerow(['Frame','id','xmin','xmax','ymin','ymax'])
writer.writerows(data)
file.close()